In [2]:
import numpy as np
import random

class Processo(object):
    def __init__(self, pnome, pio, ptam, prioridade, tempoChegada):
        self.nome = pnome
        self.io = pio  # Probabilidade de realizar I/O, inicialmente zero
        self.tam = ptam  # Timeslices totais necessários para terminar
        self.prio = prioridade  # Prioridade, não usada agora
        self.chegada = tempoChegada  # Tempo de chegada
        self.remaining_time = ptam  # Tempo de execução restante
        self.start_time = None  # Tempo em que o processo recebe a CPU pela primeira vez
        self.finish_time = None  # Tempo em que o processo termina a execução

    def roda(self, tempo, quantum=None):  # Recebe 'tempo' como parâmetro
        if self.start_time is None:
            self.start_time = tempo  # Registra a primeira vez que o processo recebe a CPU

        if random.randint(1, 100) < self.io:  # Verifica se realiza I/O
            self.remaining_time -= 1
            print(self.nome, "realizou I/O, tempo restante", self.remaining_time)
            return 1, True  # True indica que realizou I/O

        if quantum is None or self.remaining_time < quantum:
            quantum = self.remaining_time
        self.remaining_time -= quantum

        print(self.nome, "rodou por", quantum, "timeslice(s), tempo restante", self.remaining_time)
        return quantum, False  # False indica que não realizou I/O
class EscalonadorSTCF(object):
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            # Ordena os processos pelo tempo restante
            self.prontos.sort(key=lambda x: x.remaining_time)
            return self.prontos.pop(0)
        else:
            return None

class EscalonadorRoundRobin(object):
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            processo = self.prontos.pop(0)
            return processo
        else:
            return None
def executar_simulacao(escalonador_classe, quantum=2):
    # Reiniciar os processos
    for proc in procs:
        proc.remaining_time = proc.tam
        proc.start_time = None
        proc.finish_time = None

    total = sum(proc.tam for proc in procs)
    escalonador = escalonador_classe(procs)
    tempo = 0
    random.seed(0)

    while total > 0:
        p = escalonador.proximo()
        if p is not None:
            current_tempo = tempo

            if p.remaining_time < quantum:
                actual_quantum = p.remaining_time
            else:
                actual_quantum = quantum

            # Processo executa
            rodou, _ = p.roda(current_tempo, quantum=actual_quantum)
            tempo += rodou  # Atualiza 'tempo' após o processo rodar
            total -= rodou

            if p.remaining_time == 0 and p.finish_time is None:
                p.finish_time = tempo  # Define o tempo de término após atualizar 'tempo'

            if p.remaining_time > 0:
                escalonador.pronto(p)
        else:
            # Nenhum processo está pronto; avança o tempo
            tempo += 1

    # Calcular tempos médios de execução e resposta
    execution_times = []
    response_times = []

    for proc in procs:
        if proc.finish_time is not None and proc.start_time is not None:
            execution_time = proc.finish_time - proc.chegada
            response_time = proc.start_time - proc.chegada
            print(f"Processo {proc.nome}: Tempo de Execução = {execution_time}, Tempo de Resposta = {response_time}")
            execution_times.append(execution_time)
            response_times.append(response_time)
        else:
            print(f"Processo {proc.nome}: Não terminou a execução.")

    average_execution_time = sum(execution_times) / len(execution_times)
    average_response_time = sum(response_times) / len(response_times)

    print(f"\nTempo Médio de Execução: {average_execution_time}")
    print(f"Tempo Médio de Resposta: {average_response_time}")
    print("-" * 50)

# Configuração dos processos com tempos de execução variados
nprocs = 4
nomes = ['A', 'B', 'C', 'D']
chanceio = [0, 0, 0, 0]
tamanho = np.array([6, 3, 1, 7])  # Tempos de execução diferentes
procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0))

# Simulação com STCF
print("Simulação com Escalonador STCF:")
executar_simulacao(EscalonadorSTCF, quantum=1)

# Simulação com Round Robin
print("Simulação com Escalonador Round Robin:")
executar_simulacao(EscalonadorRoundRobin, quantum=2) 

Simulação com Escalonador STCF:
Simulação com Escalonador Round Robin:
A rodou por 2 timeslice(s), tempo restante 4
B rodou por 2 timeslice(s), tempo restante 1
C rodou por 1 timeslice(s), tempo restante 0
D rodou por 2 timeslice(s), tempo restante 5
A rodou por 2 timeslice(s), tempo restante 2
B rodou por 1 timeslice(s), tempo restante 0
D rodou por 2 timeslice(s), tempo restante 3
A rodou por 2 timeslice(s), tempo restante 0
D rodou por 2 timeslice(s), tempo restante 1
D rodou por 1 timeslice(s), tempo restante 0
Processo A: Tempo de Execução = 14, Tempo de Resposta = 0
Processo B: Tempo de Execução = 10, Tempo de Resposta = 2
Processo C: Tempo de Execução = 5, Tempo de Resposta = 4
Processo D: Tempo de Execução = 17, Tempo de Resposta = 5

Tempo Médio de Execução: 11.5
Tempo Médio de Resposta: 2.75
--------------------------------------------------
